In [ ]:
import os
import scipy
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "exalted-gamma-237507-cecf0237b380.json"
from extract_audio import *
from moviepy.editor import VideoFileClip, AudioFileClip
import moviepy.editor as mpe
import io
import subprocess

In [ ]:
def extractAudio(videofile,audiofile):
    command = "ffmpeg -i "+videofile+" -ab 160k -ac 1 -ar 16000 -vn "+audiofile
    subprocess.call(command, shell=True)

In [ ]:
def transcribe_file2(speech_file,language):
    """Transcribe the given audio file."""
    from google.cloud import speech
    from google.cloud.speech import enums
    from google.cloud.speech import types
    client = speech.SpeechClient()

    with io.open(speech_file, 'rb') as audio_file:
        content = audio_file.read()

    audio = types.RecognitionAudio(content=content)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code=language,
    enable_word_time_offsets=True)

#    response = client.recognize(config, audio)
    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    string=""
    operation = client.long_running_recognize(config, audio)

    ret=list()
    print('Waiting for operation to complete...')
    result = operation.result(timeout=90)
    
    for result in result.results:
        alternative = result.alternatives[0]
        string=string+alternative.transcript
        
        
        for word_info in alternative.words:
            word = word_info.word
            start_time = word_info.start_time
            end_time = word_info.end_time
            ret.append((
                word,
                start_time.seconds + start_time.nanos * 1e-9,
                end_time.seconds + end_time.nanos * 1e-9))
    return string,ret

In [4]:
from google.cloud import texttospeech

def textToSpeech(text2,language):
# Instantiates a client
    client = texttospeech.TextToSpeechClient()

# Set the text input to be synthesized
    synthesis_input = texttospeech.types.SynthesisInput(text=text2)

# Build the voice request, select the language code ("en-US") and the ssml
# voice gender ("neutral")
    voice = texttospeech.types.VoiceSelectionParams(
        language_code=language,
        ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE)

# Select the type of audio file you want returned
    audio_config = texttospeech.types.AudioConfig(
        audio_encoding=texttospeech.enums.AudioEncoding.MP3)

    # Perform the text-to-speech request on the text input with the selected
    # voice parameters and audio file type
    response = client.synthesize_speech(synthesis_input, voice, audio_config)

    # The response's audio_content is binary.
    with open('target_audio.mp3', 'wb') as out:
        # Write the response to the output file.
        out.write(response.audio_content)
        print('Audio content written to file "output.mp3"')

In [5]:
def synthesize_ssml(ssml,target):
    """Synthesizes speech from the input string of ssml.

    Note: ssml must be well-formed according to:
        https://www.w3.org/TR/speech-synthesis/

    Example: <speak>Hello there.</speak>
    """
    from google.cloud import texttospeech
    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.types.SynthesisInput(ssml=ssml)

    # Note: the voice can also be specified by name.
    # Names of voices can be retrieved with client.list_voices().
    voice = texttospeech.types.VoiceSelectionParams(
        language_code=target,
        ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE)

    audio_config = texttospeech.types.AudioConfig(
        audio_encoding=texttospeech.enums.AudioEncoding.MP3)

    response = client.synthesize_speech(input_text, voice, audio_config)

    # The response's audio_content is binary.
    with open('target_audio.mp3', 'wb') as out:
        out.write(response.audio_content)


In [6]:
def generateSsml(tuples):
    speakOpenTag="<speak>"
    speakCloseTag="</speak>"
    breakOpenTag='<break time=\"'
    breakCloseTag="s\"/>"
    ssml=speakOpenTag
    for idx,t in enumerate(tuples):
        try:
            ssml = ssml + t[0] + breakOpenTag+ str(tuples[idx+1][1]-t[2]) +breakCloseTag
        except IndexError:
            pass
    
    ssml = ssml+speakCloseTag
    return ssml
    

In [7]:
def accent_translation(file,current,target):
    extractAudio(file,"source_audio.wav")
    source_audio_text,tuples=transcribe_file2("source_audio.wav",current)
    print("source audio text",source_audio_text)
    ssml=generateSsml(tuples)
    synthesize_ssml(ssml,target)
    
    target_audio = mpe.AudioFileClip("target_audio.mp3")
    videoclip = VideoFileClip(file)
    new_clip = videoclip.set_audio(target_audio)
    new_clip.write_videofile("final_cut.mp4")

In [8]:
accent_translation("video_file.mp4",'en-US','en-IN')

Waiting for operation to complete...
source audio text faces in a question send me the third best way of starting a speech the second best way of starting a speech that shocks does more people alive today than have ever died more people alive today than have ever died does a lot of speeches that can follow on from there but every two minutes the energy reaching the Earth from the Sun is equivalent to the whole annual usage of humanity


chunk:   3%|▎         | 33/996 [00:00<00:02, 329.51it/s, now=None]

Moviepy - Building video final_cut.mp4.
MoviePy - Writing audio in %s


t:   1%|          | 10/1233 [00:00<00:12, 94.50it/s, now=None]     

MoviePy - Done.
Moviepy - Writing video final_cut.mp4



Moviepy - Done !
Moviepy - video ready final_cut.mp4
